In [ ]:
import codebase.web_scrape_functions as wsf
import codebase.analysis_functions as af
from codebase import match_data
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import utils
from utils import logger
import os
import logging
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import codebase.graphing_functions as gf
from codebase.settings import CAREERS

%load_ext autoreload
%autoreload 2
logger.handlers[1].setLevel(logging.INFO)
# logger.disabled = True
pd.get_option("display.max_columns")

In [ ]:
PLAYER_ID = '253802'
ROOT_PLAYER_ID = '303669'
WILLIAMSON_PLAYER_ID = '277906'
SPD_SMITH_ID = '267192'

In [ ]:
test_match_list = wsf.player_match_list(PLAYER_ID, _format='test')
test_match_list

In [ ]:
all_totals = af.get_cricket_totals(KOHLI_ID, matches=match_list, _type='bat', by_innings=True, is_object_id=True)

In [ ]:
import math
dismissals = {}
for inning in all_totals:
    try:
        dismissals[inning['how_out']] += 1
    except KeyError:
        dismissals[inning['how_out']] = 1

In [ ]:
# Breakdown of dismissals, we want to see sum how Kohli has got out over time and graph this.
# Need to make individuals 'lines' for each dismissal type. Lets just first start with getting Kohli's test innings and then summing dismissals.
match_list = wsf.player_match_list(KOHLI_ID)
bad_forma_matches = match_list[84:]


In [ ]:
%%script false --no-raise-error
contributions = []
for match_id in test_match_list:
    match = match_data.MatchData(match_id)
    contribution = af._cricket_totals(PLAYER_ID, match, _type='bat', by_innings=True, is_object_id=True)
    for i,inning in enumerate(contribution['bat']):
        contributions.append({**inning, **{key:contribution[key] for key in contribution.keys() if key not in ['bat', 'bowl']}, **{'inning': i}})

contributions[-5:]

In [ ]:
#innings = [inning for match in contributions for inning in match]
#innings[:5]

In [ ]:
%%script false --no-raise-error
innings_df = pd.DataFrame(contributions)
innings_df.head()
innings_df.shape

Now we will try to graph the running average on this graph. 

In [ ]:
%%script false --no-raise-error
running_average = []

total_runs = 0
out = 0

for i,innings in enumerate(zip(innings_df.runs, innings_df.not_out)):
    total_runs += innings[0]
    if innings[1] == False:
        out += 1
    try:
        running_average.append(round(total_runs/out,2))
    except ZeroDivisionError:
        running_average.append(None)

running_average

We now can map the average of the last X innings, this is the 'form guide'

In [ ]:
%%script false --no-raise-error
last_x_average = []

window_runs = 0
window_out = 0

#moving window of length x

window_size = 12

for i,innings in enumerate(zip(innings_df.runs, innings_df.not_out)):
    if i>=window_size:
        window_runs -= innings_df.runs.iloc[i-window_size]
        if innings_df.not_out.iloc[i-window_size] == False:
            window_out -= 1
    
    window_runs += innings[0]
    if innings[1] == False:
        window_out += 1
    try:
        last_x_average.append(round(window_runs/window_out,2))
    except ZeroDivisionError:
        last_x_average.append(None)

last_x_average

In [ ]:
%%script false --no-raise-error
y_range = [0, max(innings_df.runs) + 20]

fig, ax1 = plt.subplots(figsize=(18,10))
#sns.set_theme()
sns.lineplot(data = {'Average': running_average, f'Last {window_size} Innings': last_x_average}, sort = False, ax=ax1, palette='rocket')

ax1.set_ylim(y_range)

ax2 = ax1.twinx()

sns.barplot(data = innings_df, x=innings_df.index, y=innings_df.runs, alpha=0.5, ax=ax2, hue=innings_df.continent, palette='mako', dodge=False)
ax2.set_ylim(y_range)

In [ ]:
%%script false --no-raise-error
SPD_SMITH_TEST_MATCHES = wsf.player_match_list(SPD_SMITH_ID)
spd_smith_contributions = af.get_cricket_totals(SPD_SMITH_ID, SPD_SMITH_TEST_MATCHES, 'bat', True, True)

In [ ]:
%%script false --no-raise-error
# spd_smith_contributions[-5:]

In [ ]:
%%script false --no-raise-error
spd_innings = pd.DataFrame(spd_smith_contributions)
spd_innings.tail()

In [ ]:
%%script false --no-raise-error
spd_ave = af.get_running_average(SPD_SMITH_ID)
spd_recent_form = af.get_recent_form_average(SPD_SMITH_ID, window_size=12)

In [ ]:
%%script false --no-raise-error
y_range = [0, max(spd_innings.runs) + 20]

fig, ax1 = plt.subplots(figsize=(18,10))
#sns.set_theme()
sns.lineplot(data = {'Average': spd_ave, f'Last {window_size} Innings': spd_recent_form}, sort = False, ax=ax1, palette='rocket')

ax1.set_ylim(y_range)

ax2 = ax1.twinx()

sns.barplot(data = innings_df, x=spd_innings.index, y=spd_innings.runs, alpha=0.5, ax=ax2, hue=spd_innings.continent, palette='mako', dodge=False)
ax2.set_ylim(y_range)

In [ ]:
%%script false --no-raise-error
contributions = af.get_player_contributions(PLAYER_ID, test_match_list, 'bat', False, True)

In [ ]:
%%script false --no-raise-error
contributions[-5:]

In [ ]:
%%script false --no-raise-error
innings = af.get_cricket_totals(PLAYER_ID, test_match_list, 'bat', True, True)
innings[-5:]

In [ ]:
#Williamson
#af.get_career_batting_graph(277906)

In [ ]:
#Root
#af.get_career_batting_graph(303669)

In [ ]:
#Smith
# af.get_career_batting_graph(SPD_SMITH_ID)

In [ ]:
#Kohli
# af.get_career_batting_graph(PLAYER_ID, dates='2017-01-01:')

In [ ]:
#Cook
gf.get_career_batting_graph(11728, player_age='27:33')

In [ ]:
%%script false --no-raise-error

sns.set_theme()
innings_df.head()

In [ ]:
%%script false --no-raise-error
fig_dims = (15,10)
fig,ax = plt.subplots(figsize=fig_dims);
bar = sns.barplot(innings_df.index, innings_df.runs, palette='mako');
x_dates = innings_df.date.dt.strftime('%d-%m-%Y')
bar.set_xticklabels(labels=x_dates, rotation=90);

_ = plt.setp(ax.patches, linewidth=0)
ax.xaxis.set_major_locator(plt.MaxNLocator(5))

In [ ]:
top_players = wsf.read_statsguru('https://stats.espncricinfo.com/ci/engine/stats/index.html?class=1;filter=advanced;orderby=batting_average;qualmin1=30;qualval1=matches;template=results;type=batting', table_name='Overall figures')


In [ ]:
top_players = pd.DataFrame(top_players[0])

In [ ]:
all_dismissals

In [ ]:
count = 0
for dismissal in all_dismissals:
    for word in ['drive', 'cover', 'defending', 'defence', 'outside']:
        if word in dismissal[0].lower():
            if 'run out' not in dismissal[1].lower():
                print(dismissal[0])
                print()
                count += 1
                break

print(f'Dismissals that include key-words: {count}')
print(f'Dismissals that include key-words: {count/len(all_dismissals)}')

In [ ]:
len(all_dismissals)

In [ ]:
comms_pre_slump = af.get_player_contributions(KOHLI_ID, match_list[:84], 'bat', True, True)
all_every_dismissals = []
for match in comms_pre_slump:
    comms = match.iloc[-1]
    
    if comms.dismissalText not in [None, 'null', 'NaN', 'None']:
        #print(comms.dismissalText)
        all_every_dismissals.append((comms.commentTextItems, comms.dismissalText))
    else:
        print(comms.commentTextItems)

every_count = 0
for dismissal in all_every_dismissals:
    for word in ['drive', 'cover', 'defending', 'defence', 'outside']:
        if word in dismissal[0].lower():
            if 'run out' not in dismissal[1].lower():
                print(dismissal[0])
                print()
                every_count += 1
                break

print(f'Dismissals that include key-words: {every_count}')
print(f'Dismissals that include key-words: {every_count/len(all_every_dismissals)}')

In [ ]:
def search_for_keywords(text_items, keywords = [], exclude_words = [], return_matching = False):
    count = 0
    matching = []
    for text in text_items:
        for word in keywords:
            if word in text.lower():
                exclude = False
                for e_word in exclude_words:
                    if e_word in text.lower():
                        exclude = True
                        break
                if exclude:
                    break
                count += 1
                matching.append(text)
                break
    if return_matching:
        return count, matching
    return count

In [ ]:
all_dismissals = [dismissal[0] + ', ' + dismissal[1] for dismissal in all_dismissals]

In [ ]:
count, matching = search_for_keywords(all_dismissals, ['drive', 'cover', 'defending', 'defence', 'outside'], exclude_words=['run out'], return_matching=True)

In [ ]:
count_oe, matching_oe = search_for_keywords(all_dismissals, ['outside off', 'outside edge', 'cover drive', 'drive'], exclude_words=['inside', 'run out'], return_matching=True)

In [ ]:
def get_dismissal_descriptions(commentary):
    all_dismissals = []
    for match in commentary:
        comms = match.iloc[-1]
        
        if comms.dismissalText not in [None, 'null', 'NaN', 'None']:
            #print(comms.dismissalText)
            all_dismissals.append((comms.commentTextItems, comms.dismissalText))
        #else:
            #print(comms.commentTextItems)
    return all_dismissals

In [ ]:
top_players.Player = [wsf.player_id_from_link(player, playername=False)[1] for player in top_players.Player]

In [ ]:
top_players[top_players.Player == '267192']

In [ ]:
%%script false --no-raise-error
match_list = []
for player in top_players.Player:
    match_list += wsf.player_match_list(player)

In [ ]:
# set([int(id) for id in match_list])

In [ ]:
# len(match_list)

In [ ]:
%%script false --no-raise-error
recent_form = {}
for player in [PLAYER_ID, SPD_SMITH_ID, ROOT_PLAYER_ID, WILLIAMSON_PLAYER_ID]:
    player_match_list = wsf.player_match_list(player)
    player_innings_df = af.get_cricket_totals(player, player_match_list, _type='bat', by_innings=True, is_object_id=True)
    player_innings_df = pd.DataFrame(player_innings_df)
    recent_form[player] = af.calculate_recent_form_average(player_innings_df)

In [ ]:
%%script false --no-raise-error
top_players.Player

In [ ]:
all_recent_form = utils.load_data('all_recent_form','dict',file_ext='p')

In [ ]:
all_running_ave  = utils.load_data('all_running_ave','dict',file_ext='p')

In [ ]:
all_running_ave.keys()

In [ ]:
problem_players = []
for player in all_recent_form:
    try:
        [int(x) for x in all_recent_form[player]]
    except (ValueError, TypeError):
        problem_players.append(player)

problem_players
        

In [ ]:
full_df = af.normalized_career_length(all_recent_form)

In [ ]:
import math
dismissals = {}
for inning in all_totals:
    try:
        dismissals[inning['how_out']] += 1
    except KeyError:
        dismissals[inning['how_out']] = 1

In [ ]:
dismissals

In [ ]:
test = [{'how_out':'caught'},{'how_out':'caught'},{'how_out':'caught'},{'how_out':'caught'},{'how_out':'caught'},{'how_out':'caught'}]

In [ ]:
dismissal_histories = {}
for i, inning in enumerate(all_totals):
    try:
        dismissal_histories[inning['how_out']] += [dismissal_histories[inning['how_out']][-1]+1]
    except KeyError:
        dismissal_histories[inning['how_out']] = [0]*(i) + [1]        
    for d in dismissal_histories:
        if d != inning['how_out']:
            dismissal_histories[d] += [dismissal_histories[d][-1]]

In [ ]:
fig, ax1 = plt.subplots(figsize=(18,10)) 
sns.lineplot(data = dismissal_histories, ax=ax1, sort = False, palette='rocket', linewidth=2)

In [ ]:
gf.get_career_batting_graph(KOHLI_ID)

In [ ]:
def graph_dismissals(player_id, match_list = None, dates = None):
    if not match_list:
        match_list = wsf.player_match_list(player_id, dates=dates)
    all_totals = af.get_cricket_totals(player_id, matches=match_list, _type='bat', by_innings=True, is_object_id=True)
    dismissal_histories = {}
    for i, inning in enumerate(all_totals):
        try:
            dismissal_histories[inning['how_out']] += [dismissal_histories[inning['how_out']][-1]+1]
        except KeyError:
            dismissal_histories[inning['how_out']] = [0]*(i) + [1]        
        for d in dismissal_histories:
            if d != inning['how_out']:
                dismissal_histories[d] += [dismissal_histories[d][-1]]
    
    fig, ax1 = plt.subplots(figsize=(18,10)) 
    sns.lineplot(data = dismissal_histories, ax=ax1, sort = False, palette='rocket', linewidth=2)

In [ ]:
graph_dismissals(KOHLI_ID, match_list=match_list[-23:])

In [ ]:
match_list[-23]

In [ ]:
gf.get_career_batting_graph(KOHLI_ID, dates='2020-02-21:')

In [ ]:
# Lets see the text of Virat Kohli's actually dismissals. So get matches, get the comms from the matches and then pull the commentary text from there. 

In [ ]:
# Lets only analyse Kohli's recent "bad form"
kohli_bad_form_comms = af.get_player_contributions(KOHLI_ID, bad_forma_matches, 'bat', True, True)

In [ ]:
all_dismissals = []
for match in kohli_bad_form_comms:
    comms = match.iloc[-1]
    
    if comms.dismissalText not in [None, 'null', 'NaN', 'None']:
        print(comms.dismissalText)
        all_dismissals.append((comms.commentTextItems, comms.dismissalText))
    else:
        print(comms.commentTextItems)

In [ ]:
all_dismissals

In [ ]:
count = 0
for dismissal in all_dismissals:
    for word in ['drive', 'cover', 'defending', 'defence', 'outside']:
        if word in dismissal[0].lower():
            if 'run out' not in dismissal[1].lower():
                print(dismissal[0])
                print()
                count += 1
                break

print(f'Dismissals that include key-words: {count}')
print(f'Dismissals that include key-words: {count/len(all_dismissals)}')

In [ ]:
len(all_dismissals)

In [ ]:
comms_pre_slump = af.get_player_contributions(KOHLI_ID, match_list[:84], 'bat', True, True)
all_every_dismissals = []
for match in comms_pre_slump:
    comms = match.iloc[-1]
    
    if comms.dismissalText not in [None, 'null', 'NaN', 'None']:
        #print(comms.dismissalText)
        all_every_dismissals.append((comms.commentTextItems, comms.dismissalText))
    else:
        print(comms.commentTextItems)

every_count = 0
for dismissal in all_every_dismissals:
    for word in ['drive', 'cover', 'defending', 'defence', 'outside']:
        if word in dismissal[0].lower():
            if 'run out' not in dismissal[1].lower():
                print(dismissal[0])
                print()
                every_count += 1
                break

print(f'Dismissals that include key-words: {every_count}')
print(f'Dismissals that include key-words: {every_count/len(all_every_dismissals)}')

In [ ]:
def search_for_keywords(text_items, keywords = [], exclude_words = [], return_matching = False):
    count = 0
    matching = []
    for text in text_items:
        for word in keywords:
            if word in text.lower():
                exclude = False
                for e_word in exclude_words:
                    if e_word in text.lower():
                        exclude = True
                        break
                if exclude:
                    break
                count += 1
                matching.append(text)
                break
    if return_matching:
        return count, matching
    return count

In [ ]:
all_dismissals = [dismissal[0] + ', ' + dismissal[1] for dismissal in all_dismissals]

In [ ]:
count, matching = search_for_keywords(all_dismissals, ['drive', 'cover', 'defending', 'defence', 'outside'], exclude_words=['run out'], return_matching=True)

In [ ]:
count_oe, matching_oe = search_for_keywords(all_dismissals, ['outside off', 'outside edge', 'cover drive', 'drive'], exclude_words=['inside', 'run out'], return_matching=True)

In [ ]:
def get_dismissal_descriptions(commentary):
    all_dismissals = []
    for match in commentary:
        comms = match.iloc[-1]
        
        if comms.dismissalText not in [None, 'null', 'NaN', 'None']:
            #print(comms.dismissalText)
            all_dismissals.append((comms.commentTextItems, comms.dismissalText))
        #else:
            #print(comms.commentTextItems)
    return all_dismissals

In [ ]:
top_four_comms = {}
for player in ['253802', 303669, 277906, 267192]:
    inning = utils.load_data(player, 'career_innings_stats', data_folder=CAREERS)
    innings_df = pd.read_json(inning)
    top_four_comms[player] = innings_df

In [ ]:
top_four_stats = af.apply_aggregate_func_to_list(['253802', '303669', '277906', '267192'], [af.calculate_recent_form_average, af.calculate_running_average], dates='2020-02-21:', disable_logging=False)

In [ ]:
top_four_stats.keys()

In [ ]:
top_four_comms = {}
for player in ['253802', '303669', '277906', '267192']:
    inning = utils.load_data(player, 'career_innings_stats', data_folder=CAREERS)
    innings_df = pd.read_json(inning)
    top_four_comms[player] = innings_df[innings_df.date >= '2020-02-21']

In [ ]:
gf.graph_career_batting_summary(top_four_stats['calculate_recent_form_average'], top_four_stats['calculate_running_average'], top_four_comms)

In [ ]:
results = []
for player in ['253802', '303669', '277906', '267192']:
    matches = wsf.player_match_list(player)
    player_contr = af.get_player_contributions(player, matches=matches, _type='bat', by_innings=True, is_object_id=True)
    dismissals = get_dismissal_descriptions(player_contr)
    dismissals = [dismissal[0] + ', ' + dismissal[1] for dismissal in dismissals]
    count, matching = search_for_keywords(dismissals, ['drive', 'cover', 'defending', 'defence', 'outside'], exclude_words=['run out'], return_matching=True)
    count_oe, matching_oe = search_for_keywords(dismissals, ['outside off', 'outside edge', 'cover drive', 'drive'], exclude_words=['inside', 'run out'], return_matching=True)
    results.append((count, count_oe, len(dismissals)))
    # results.append(count_oe)
    # results.append(f"For player {player}, there were {count}/{len(dismissals)} that include the specified key-words: {['drive', 'cover', 'defending', 'defence', 'outside']}")
    # results.append(f"For player {player}, there were {count_oe}/{len(dismissals)} that include the specified key-words: {['outside off', 'outside edge', 'cover drive', 'drive']} ")

for result in results:
    print(f"For player {player}, there were {result[0]}/{result[2]} that include the specified key-words: {['drive', 'cover', 'defending', 'defence', 'outside']}")
    print(f"For player {player}, there were {result[1]}/{result[2]} that include the specified key-words: {['outside off', 'outside edge', 'cover drive', 'drive']} ")
    

In [ ]:
for result in results:
    for i in [0,1]:
        print(result[i]/result[2])

In [ ]:
top_players = [PLAYER_ID, SPD_SMITH_ID, ROOT_PLAYER_ID, WILLIAMSON_PLAYER_ID]
gf.graph_multi_player_batting_careers(top_players, dates ="2020-02-21:")

In [ ]:
gf.graph_multi_player_batting_careers(top_players, dates ="2020-02-21:")

In [ ]:
top_indians_table = "https://stats.espncricinfo.com/ci/engine/stats/index.html?class=1;filter=advanced;orderby=batting_average;qualmin1=20;qualval1=matches;team=6;template=results;type=batting"
top_indians = wsf.read_statsguru(top_indians_table, table_name='Overall figures')

In [ ]:
top_indians = top_indians[0]
top_indians.Player = [wsf.player_id_from_link(val, playername=False)[1] for val in top_indians.Player]

In [ ]:
top_indians

In [ ]:
top_20_indians = [player for player in top_indians.Player[:20]]

In [ ]:
top_20_indians

In [ ]:
gf.graph_multi_player_batting_careers(top_20_indians)

In [ ]:
all_recent_form_indians  = utils.load_data('all_recent_form_indians','dict',file_ext='p')
all_innings_indians  = utils.load_data('all_running_ave_indians','dict',file_ext='p')


In [ ]:
all_running_ave_indians = {}
for player in all_innings_indians:
    all_running_ave_indians[player] = af.calculate_running_average(all_innings_indians[player])

In [ ]:
gf.graph_career_batting_summary(all_recent_form_indians, all_running_ave_indians, all_innings_indians)

In [ ]:
results = []
for player in ['253802', '303669', '277906', '267192']:
    matches = wsf.player_match_list(player)
    player_contr = af.get_player_contributions(player, matches=matches, _type='bat', by_innings=True, is_object_id=True)
    dismissals = get_dismissal_descriptions(player_contr)
    dismissals = [dismissal[0] + ', ' + dismissal[1] for dismissal in dismissals]
    count, matching = search_for_keywords(dismissals, ['drive', 'cover', 'defending', 'defence', 'outside'], exclude_words=['run out'], return_matching=True)
    count_oe, matching_oe = search_for_keywords(dismissals, ['outside off', 'outside edge', 'cover drive', 'drive'], exclude_words=['inside', 'run out'], return_matching=True)
    results.append((count, count_oe, len(dismissals)))
    # results.append(count_oe)
    # results.append(f"For player {player}, there were {count}/{len(dismissals)} that include the specified key-words: {['drive', 'cover', 'defending', 'defence', 'outside']}")
    # results.append(f"For player {player}, there were {count_oe}/{len(dismissals)} that include the specified key-words: {['outside off', 'outside edge', 'cover drive', 'drive']} ")

for result in results:
    print(f"For player {player}, there were {result[0]}/{result[2]} that include the specified key-words: {['drive', 'cover', 'defending', 'defence', 'outside']}")
    print(f"For player {player}, there were {result[1]}/{result[2]} that include the specified key-words: {['outside off', 'outside edge', 'cover drive', 'drive']} ")
    

In [ ]:
for result in results:
    for i in [0,1]:
        print(result[i]/result[2])